# write nextflow config

In [1]:
%%writefile environment.yml
name: midrc_env
channels:
  - defaults
dependencies:
  - pip
  - pip:
    - pyyaml==5.3.1
    - pydicom
    - pillow
    - gen3
        

Overwriting environment.yml


In [2]:
%%writefile nextflow.config

conda.enabled = true

process {
    conda {
        envCreate = true
        envName = 'midrc_env'
        // If the environment does not exist, Nextflow will create it using the specified YAML file
        envFile = 'environment.yml'
        
    }
}

workDir = 'sdk_data'

Overwriting nextflow.config


# Pull some midrc guids to test and move dcm files to midrc_dicom directory

In [3]:
%%writefile download_data.py

import gen3
import os

output_dir = 'data'

guids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855',
         'dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb',
         'dg.MD1R/e604979a-c71b-4ec6-b8a0-959837b86384',
         'dg.MD1R/b5cee98d-46ff-4438-aa00-90727a383340',
         'dg.MD1R/8a5a5579-7925-432d-a614-3ed208f1c182',
         'dg.MD1R/33034812-47f3-4c0e-b60b-fa7a2a04ecda',
         'dg.MD1R/5ca987c5-c660-4785-a67d-a3424cc8ec6e',
         'dg.MD1R/44148117-1858-49ef-b30f-d239abfaff80',
         'dg.MD1R/9ea205e8-a774-4318-a323-95eadda9bc5c',
         'dg.MD1R/09ece36f-a0fa-48e8-8fc2-62110eaae570']

for guid in guids:
    print('pulling guid {}'.format(guid))
    os.system(f"gen3 drs-pull object {guid}")


Overwriting download_data.py


In [4]:
%%writefile convert_dcm_to_png.py

import pydicom
import argparse
import numpy as np
from PIL import Image
import os

def main(dicom_inputs):
    for dicom_file in dicom_inputs:
        png_out = dicom_file.split('.dcm')[0] + '.png'
        dicom_dataset = pydicom.dcmread(dicom_file)
        transformed_image = dicom_dataset.pixel_array.astype(float)
        scaled_image = np.uint8((np.maximum(transformed_image, 0) / transformed_image.max()) * 255.0)
        final_image = Image.fromarray(scaled_image)
        final_image.save(png_out)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='convert dicom to png')
    parser.add_argument('--dicom-inputs', dest='dicom_inputs', nargs = '+', required=True)
    args = parser.parse_args()

    main(args.dicom_inputs)

Overwriting convert_dcm_to_png.py


In [5]:
%%writefile main.nf
#!/usr/bin/env nextflow

process DownloadData {
    
    conda './environment.yml' 
    
    label 'download_data'
    
    tag 'initial_task'
    
    output:
    stdout emit: download_data_log
    path('**/*.dcm'), emit: dicom_image
    
    script:
    """
    python3 ${baseDir}/download_data.py
    """
}

process PublishData {
    
    conda './environment.yml' 
    
    label 'publish_data'
    
    publishDir "${baseDir}/dicom_files", mode: 'copy'
    
    input:
    path(dicom_image) 
    
    output:
    stdout emit: convert_to_png_log
    
    script:
    """
    echo $dicom_image
    """
}


process convert_dcm_to_png {
    
    conda './environment.yml' 
    
    label 'dcm2png'
    
    input:
    path(dicom_image) 
    
    output:
    stdout emit: convert_to_png_log
    path('*.png'), emit: png_image
    
    script:
    """
    python3 ${baseDir}/convert_dcm_to_png.py --dicom-inputs $dicom_image
    """
}


// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    DownloadData()
    PublishData(DownloadData.out.dicom_image)
    convert_dcm_to_png(DownloadData.out.dicom_image)
}

Overwriting main.nf


In [ ]:
!nextflow run main.nf

N E X T F L O W  ~  version 22.10.6
Launching `main.nf` [clever_stonebraker] DSL2 - revision: 0f0c4e1aa0
[-        ] process > DownloadData       -
[-        ] process > PublishData        -
[-        ] process > convert_dcm_to_png -

[-        ] process > DownloadData       -
[-        ] process > PublishData        -
[-        ] process > convert_dcm_to_png -
Creating env using conda: ./environment.yml [cache /home/jovyan/pd/sdk_data/conda/midrc_env-5737b5ae07a80b35c557e66cbd226316]

executor >  local (1)
[15/140ca0] process > DownloadData (initial_task) [  0%] 0 of 1
[-        ] process > PublishData                 -
[-        ] process > convert_dcm_to_png          -

